# Manage references

In [1]:
import os
from pathlib import Path
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs

In [4]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'iucnecoadm'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))


In [40]:
conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)

In [41]:
# we added a column to the table
upd = """ALTER TABLE ref_list ADD COLUMN ref_markdown text;"""
# now we can update this column with existing content:
upd = """UPDATE ref_list set ref_markdown=ref_cite
WHERE ref_cite like '% **%** %' AND ref_markdown is NULL;"""
cur.execute(upd)
upd = """UPDATE ref_list set ref_markdown=ref_cite
WHERE ref_cite like '% **%**.%' AND ref_markdown is NULL;"""
cur.execute(upd)


In [42]:
print(cur.rowcount)


16


In [43]:
upd = """UPDATE ref_list set ref_markdown=CONCAT(author_list,
' (',date,') *',title,'* **',container_title,'** ', post_title, 
' [DOI:',doi,'](http:dx.doi.org/',doi,')') 
WHERE author_list IS NOT NULL
AND date IS NOT NULL
AND title IS NOT NULL
AND container_title IS NOT NULL
AND doi IS NOT NULL
AND ref_markdown is NULL;"""
cur.execute(upd)
print(cur.rowcount)

291


In [44]:
upd = """UPDATE ref_list set ref_markdown=CONCAT(author_list,
' (',date,') **',container_title,'** ', post_title, 
' [DOI:',doi,'](http:dx.doi.org/',doi,')') 
WHERE author_list IS NOT NULL
AND date IS NOT NULL
AND title IS NULL
AND container_title IS NOT NULL
AND doi IS NOT NULL
AND ref_markdown is NULL;"""
cur.execute(upd)
print(cur.rowcount)

25


In [45]:
upd = """UPDATE ref_list set ref_markdown=CONCAT(author_list,
' (',date,') *',title,'* **',container_title,'** ', post_title) 
WHERE author_list IS NOT NULL
AND date IS NOT NULL
AND title IS NOT NULL
AND container_title IS NOT NULL
AND ref_markdown is NULL;"""
cur.execute(upd)
print(cur.rowcount)

139


In [46]:
upd = """UPDATE ref_list set ref_markdown=CONCAT(author_list,
' (',date,') **',container_title,'** ', post_title) 
WHERE author_list IS NOT NULL
AND date IS NOT NULL
AND title IS NULL
AND container_title IS NOT NULL
AND ref_markdown is NULL;"""
cur.execute(upd)
print(cur.rowcount)

67


In [47]:
conn.commit()

In [48]:

cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.
